In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import pandas_datareader.data as web
import datetime

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
df = pd.read_csv("/kaggle/input/bitcoin-historical-data/bitstampUSD_1-min_data_2012-01-01_to_2021-03-31.csv") 

In [ ]:
df.head()
df.info()

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
# converting dates/time columns into a datetime object
df["Period"] = pd.to_datetime(df["Timestamp"],unit='s')
# set the new datetime column as the index
df = df.set_index("Period")
# create new columns from datetime index
df["year"] = df.index.year
df["month"] = df.index.month
# new dataframe
df.head()

In [ ]:
df = df.interpolate()

In [ ]:
x = df.groupby(["year", "month"])["Weighted_Price"].mean()
df_wide = x.unstack()
df_wide.head()


In [ ]:
df["Weighted_Price"].plot()
plt.show()

In [ ]:
# plotting with confidence intervals 
sns.lineplot(x="year", y="Weighted_Price", data=df)

In [ ]:
ewma = pd.Series.ewm
price = df["Close"]

n = 14

def RSI(series,n):    
    delta = series.diff()
    u = delta * 0 
    d = u.copy()
    i_pos = delta > 0
    i_neg = delta < 0
    u[i_pos] = delta[i_pos]
    d[i_neg] = delta[i_neg]
    rs = ewma(u, span=n).mean() / ewma(d, span=n).mean()
    return 100 - 100 / (1 + rs)

dfrsi = RSI(price,n)
dfrsi.plot()
plt.show()

In [ ]:
def calc_indexes(df, last_values = 14)-> dict:
    '''
    Calculates RSI in last_values observed
    RSI below 30 is undervalued
    RSI above 70 is overvalued
    get_all_tickers => https://github.com/shilewenuw/get_all_tickers
    '''
    from statistics import mean
    # This list holds the closing prices of a stock
    prices = crypto_history
    i = 0
    upPrices, downPrices = [], []
    #  Loop to hold up and down price movements
    while i < len(prices):
        if i == 0:
            upPrices.append(0)
            downPrices.append(0)
        else:
            if (prices[i] - prices[i-1]) > 0 :
                upPrices.append(prices[i]-prices[i-1])
                downPrices.append(0)
            else:
                downPrices.append(prices[i]-prices[i-1])
                upPrices.append(0)
        i += 1
    x = 0
    avg_gain, avg_loss = [], []
    #  Loop to calculate the average gain and loss
    while x < len(upPrices):
        if x < last_values+1:
            avg_gain.append(0)
            avg_loss.append(0)
        else:
            sumGain, sumLoss = 0, 0
            y = x-last_values
            while y <= x:
                sumGain += upPrices[y]
                sumLoss += downPrices[y]
                y += 1
            try:
                gain = sumGain/last_values
            except Exception as e:
                gain = 0
            try:
                loss =sumLoss/last_values
            except Exception as e:
                loss= 0
            avg_gain.append( gain)
            avg_loss.append( abs( loss ) )
        x += 1
    #  Loop to calculate RSI and RS
    p = 0
    RS, RSI = [], []
    while p < len(prices):
        if p < last_values+1:
            RS.append(0)
            RSI.append(0)
        else:
            RSvalue = (avg_gain[p]/avg_loss[p])
            RS.append(RSvalue)
            RSI.append(100 - (100/(1+RSvalue)))
        p+=1
    #  Creates the csv for each stock's RSI and price movements
    df_dict = {
        'Prices' : prices,
        'upPrices' : upPrices,
        'downPrices' : downPrices,
        'AvgGain' : avg_gain,
        'AvgLoss' : avg_loss,
        'RS' : RS,
        'RSI' : RSI
    }
    df = pd.DataFrame(df_dict, columns = ['Prices', 'upPrices', 'downPrices', 'AvgGain','AvgLoss', 'RS', "RSI"])
    if df.empty:
        return None
    else:
        return df
def calc_indexes(crypto_history, last_values = 14)-> dict:
    '''
    Calculates RSI in last_values observed
    RSI below 30 is undervalued
    RSI above 70 is overvalued
    get_all_tickers => https://github.com/shilewenuw/get_all_tickers
    '''
    from statistics import mean
    # This list holds the closing prices of a stock
    prices = crypto_history
    i = 0
    upPrices, downPrices = [], []
    #  Loop to hold up and down price movements
    while i < len(prices):
        if i == 0:
            upPrices.append(0)
            downPrices.append(0)
        else:
            if (prices[i] - prices[i-1]) > 0 :
                upPrices.append(prices[i]-prices[i-1])
                downPrices.append(0)
            else:
                downPrices.append(prices[i]-prices[i-1])
                upPrices.append(0)
        i += 1
    x = 0
    avg_gain, avg_loss = [], []
    #  Loop to calculate the average gain and loss
    while x < len(upPrices):
        if x < last_values+1:
            avg_gain.append(0)
            avg_loss.append(0)
        else:
            sumGain, sumLoss = 0, 0
            y = x-last_values
            while y <= x:
                sumGain += upPrices[y]
                sumLoss += downPrices[y]
                y += 1
            try:
                gain = sumGain/last_values
            except Exception as e:
                gain = 0
            try:
                loss =sumLoss/last_values
            except Exception as e:
                loss= 0
            avg_gain.append( gain)
            avg_loss.append( abs( loss ) )
        x += 1
    #  Loop to calculate RSI and RS
    p = 0
    RS, RSI = [], []
    while p < len(prices):
        if p < last_values+1:
            RS.append(0)
            RSI.append(0)
        else:
            RSvalue = (avg_gain[p]/avg_loss[p])
            RS.append(RSvalue)
            RSI.append(100 - (100/(1+RSvalue)))
        p+=1
    #  Creates the csv for each stock's RSI and price movements
    df_dict = {
        'Prices' : prices,
        'upPrices' : upPrices,
        'downPrices' : downPrices,
        'AvgGain' : avg_gain,
        'AvgLoss' : avg_loss,
        'RS' : RS,
        'RSI' : RSI
    }
    df = pd.DataFrame(df_dict, columns = ['Prices', 'upPrices', 'downPrices', 'AvgGain','AvgLoss', 'RS', "RSI"])
    if df.empty:
        return None
    else:
        return df

In [ ]:
result = calc_indexes(df.Close)

In [ ]:
result.RSI[-100:].plot()
plt.axhline(y=30, color='r', linestyle='-')
plt.axhline(y=70, color='r', linestyle='-')
plt.show() 

